# PPP FOIA Data Cleaning

Data files can be downloaded from https://data.sba.gov/dataset/ppp-foia

In [1]:
import json
import os
from pathlib import Path

import pandas as pd
from tqdm import tqdm
from ydata_profiling import ProfileReport
import uszipcode

search = uszipcode.SearchEngine()

C:\Users\knugg\Projects\ppp-loans-ds4a\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Combine & Load Data
The full data set contains over 10M items. Use nrows to reduce loaded data for faster processing during exploration if needed.

In [2]:
nrows = None # Integer number of rows to load from each file. Use None to read in all rows.
data_dir = Path('../data/PPP-FOIA/')
cleaned_data_dir = data_dir / 'cleaned'
cleaned_data_dir.mkdir(exist_ok=True)

In [3]:
# Running this with all data uses ~9 Gb of RAM
df = None
low_memory = False # Allows checking entire file to decide dtypes (removes warning)
filenames = os.listdir(data_dir)
filenames = [f for f in filenames if f.endswith('.csv')]
print(f'Loading {len(filenames)} data files.')
for filename in tqdm(filenames):
    if df is None:
        df = pd.read_csv(data_dir / filename, nrows=nrows, low_memory=low_memory)
    else:
        df = pd.concat([df, pd.read_csv(data_dir / filename, nrows=nrows, low_memory=low_memory)])

Loading 13 data files.


100%|███████████████████████████████████████████████████████████████████████████| 13/13 [02:14<00:00, 10.32s/it]


In [4]:
unique_lender = len(df.OriginatingLender.unique())
unique_lender

4676

In [5]:
total_loans = len(df)
top_100 = sum(df['OriginatingLender'].value_counts()[:100])
top_1000 = sum(df['OriginatingLender'].value_counts()[:1000])
top_half = sum(df['OriginatingLender'].value_counts()[:(int(.8*unique_lender))])
print(f'{100*top_100/total_loans}% of loans accounted for by top 100 banks')
print(f'{100*top_1000/total_loans}% of loans accounted for by top 1000 banks')
print(f'{100*top_half/total_loans}% of loans accounted for by top 80% of banks')

66.05696535265773% of loans accounted for by top 100 banks
88.72517064083296% of loans accounted for by top 1000 banks
99.5221576767106% of loans accounted for by top 80% of banks


In [6]:
# Create a backup of full data for quick reload during exploration
df_bu = df

## Load Column Descriptions:

In [7]:
column_names_df = pd.read_excel(data_dir / 'ppp-data-dictionary.xlsx')
column_names_df = column_names_df.rename(columns=lambda x: x.replace(' ', ''))

# Data Cleaning:

## Drop Columns
List each column to drop and perform operation before moving forward

In [8]:
drop_columns = [
    'SBAOfficeCode', # Originating office. Not used
    'ProcessingMethod', # PPP or PPS? Not used
    'SBAGuarantyPercentage', # Every value is 100%
    'FranchiseName', # 98.7% Missing value
    'ServicingLenderLocationID', # Not concerned with servicing lender
    'ServicingLenderName',
    'ServicingLenderAddress',
    'ServicingLenderCity',
    'ServicingLenderState',
    'ServicingLenderZip',
    'HubzoneIndicator', # Not used
    'LMIIndicator', # What is it? Not used
    'BusinessAgeDescription', # Not used
    'ProjectCity', # Project data. Not used
    'ProjectCountyName',
    'ProjectState',
    'ProjectZip',
    'CD', # Project Congressional District. Not used
    'Race', # Not used
    'Ethnicity', # Not used
    'UTILITIES_PROCEED', # PROCEED Data not used
    'PAYROLL_PROCEED',
    'MORTGAGE_INTEREST_PROCEED',
    'RENT_PROCEED',
    'REFINANCE_EIDL_PROCEED',
    'HEALTH_CARE_PROCEED',
    'DEBT_INTEREST_PROCEED',
    'OriginatingLenderLocationID', # Not used
    'Gender', # Not used
    'Veteran', # Not used
    'NonProfit', # Not used    
]

In [9]:
df = df.drop(columns=drop_columns, axis=1)
df.columns

Index(['LoanNumber', 'DateApproved', 'BorrowerName', 'BorrowerAddress',
       'BorrowerCity', 'BorrowerState', 'BorrowerZip', 'LoanStatusDate',
       'LoanStatus', 'Term', 'InitialApprovalAmount', 'CurrentApprovalAmount',
       'UndisbursedAmount', 'RuralUrbanIndicator', 'JobsReported', 'NAICSCode',
       'BusinessType', 'OriginatingLender', 'OriginatingLenderCity',
       'OriginatingLenderState', 'ForgivenessAmount', 'ForgivenessDate'],
      dtype='object')

# Clean Reamining Columns:

## LoanNumber
100% distinct. No missing values. No actions required.

## DateApproved
No missing values. Convert to date type.

In [10]:
df.DateApproved = pd.to_datetime(df.DateApproved)

## BorrowerName
58 Missing values. Reviewed the data and other fields were provided. Changing name to 'name not provided' to clear the nulls. Converting to categorical.

In [11]:
df.BorrowerName = df.BorrowerName.fillna('missing borrower name')
df.BorrowerName = pd.Categorical(df.BorrowerName)

In [12]:
df[df.BorrowerName.isnull()]

,LoanNumber,DateApproved,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,...,UndisbursedAmount,RuralUrbanIndicator,JobsReported,NAICSCode,BusinessType,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,ForgivenessAmount,ForgivenessDate


## BorrowerAddress
214 missing values. Of these, 191 do not provide city or zip. Dropping 191 rows, filling the rest with 'missing address'

In [13]:
df = df.dropna(subset=['BorrowerAddress', 'BorrowerCity', 'BorrowerZip'], how='all')

In [14]:
df.BorrowerAddress = df.BorrowerAddress.fillna('missing address')

In [15]:
df[df.BorrowerAddress.isnull()]

,LoanNumber,DateApproved,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,LoanStatusDate,LoanStatus,Term,...,UndisbursedAmount,RuralUrbanIndicator,JobsReported,NAICSCode,BusinessType,OriginatingLender,OriginatingLenderCity,OriginatingLenderState,ForgivenessAmount,ForgivenessDate


## BorrowerCity
Using uszipcode library to fill in missing city. If the zip and city are not available, we do not know the location of the bank. Check for bank names.

In [16]:
def zip2city(zip_code):
    if pd.isnull(zip_code):
        return None
    else:
        return search.by_zipcode(zip_code).major_city

In [17]:
df.loc[df.BorrowerCity.isnull() ,'BorrowerCity'] = df[df.BorrowerCity.isnull()]['BorrowerZip'].apply(zip2city)

In [18]:
df.BorrowerCity.isnull().sum()

0

## BorrowerState
Using uszipcode library to fill in missing states.

In [19]:
def zip2state(zip_code):
    # Only use 5 digit zip
    if pd.isnull(zip_code):
        return None
    zip_code = zip_code.split('-')[0]
    return search.by_zipcode(zip_code).state_abbr

In [20]:
df.loc[df.BorrowerState.isnull(), 'BorrowerState'] = df[df.BorrowerState.isnull()].BorrowerZip.apply(zip2state)

In [21]:
df.BorrowerState.isnull().sum()

0

## BorrowerZip
5 missing values remain. Fill in with uszipcode search by city, state.

In [22]:
city_state_w_null_zip = list(zip(df[df.BorrowerZip.isnull()].BorrowerCity, df[df.BorrowerZip.isnull()].BorrowerState))
# df.BorrowerZip[df.BorrowerZip.isnull()] = [search.by_city_and_state(cs[0], cs[1]) for cs in city_state_w_null_zip]
zipcodes = [search.by_city_and_state(cs[0], cs[1])[0].zipcode for cs in city_state_w_null_zip]
df.loc[df.BorrowerZip.isnull(), 'BorrowerZip'] = zipcodes

In [23]:
df.BorrowerZip.isnull().sum()

0

## Loan Status Date:
Missing 565782 values (4.9% of values). Every loan missin Loan Status Date has a Loan Status of "Exemption 4." Loans considered exempt, were those whose status was protected by FOIA’s Exemption 4, which is specifically intended to protect “submitters who are required to furnish commercial or financial information to the government by safeguarding them from the competitive disadvantages that could result from disclosure.”

Leaving the missing data as null

In [24]:
# df.loc[df.LoanStatusDate.isnull(), 'LoanStatusDate'] = pd.Timestamp.now()

In [25]:
df.LoanStatusDate.isnull().sum()

565782

## Loan Status:
No missing values

## Term
No issues.

## Initial Approval Amount
20 values less than or equal to zero. Each of these loans had an amount for Current Approval Amount and had 0 for Undisbursed Amount. Setting the value of Initial Approval Amount to Current Approval Amount.

In [26]:
df.loc[df.InitialApprovalAmount <= 0.0, 'InitialApprovalAmount'] = df['CurrentApprovalAmount'][df.InitialApprovalAmount <= 0.0]

In [27]:
len(df[df.InitialApprovalAmount <= 0.0])

0

## Current Approval Amount
No missing values.

## Undisbursed Amount
Missing 1171 values. 99.9% of values in this field are zero. Setting missing values to zero (already skew).

In [28]:
df.loc[df.UndisbursedAmount.isnull(), 'UndisbursedAmount'] = 0

In [29]:
len(df[df.UndisbursedAmount.isnull()])

0

## Jobs Reported
One negative value and 8 missing. 210 Zeros (likely people not counting themselves). Setting missing / negative values to 1.

In [30]:
df.loc[df.JobsReported.isnull(), 'JobsReported'] = 1
df.loc[df.JobsReported < 0, 'JobsReported'] = 1

## NAICS Code
Missing 132292 (1.2%) values. Converting to datatype string. Leaving null fields.

In [31]:
df.NAICSCode = df.NAICSCode.astype(str)

## Business Type
Missing 2233 values. Many business names contain "LLC" or "INC" updating these. Remaining 950 null values converted to 'Other'

In [32]:
df.loc[df.BusinessType.isnull() & df.BorrowerName.str.contains('INC'), 'BusinessType'] = 'Corporation'
df.loc[df.BusinessType.isnull() & df.BorrowerName.str.contains('LLC'), 'BusinessType'] = 'Limited  Liability Company(LLC)'
df.loc[df.BusinessType.isnull(), 'BusinessType'] = 'Other / Unknown'

In [33]:
print(len(df[df.BusinessType.isnull()]))

0


## Originating Lender (Lender, City, State)
No missing values.

## Forgiveness Amount
Missing 914809 values. Setting forgiveness amount to zero for missing values.

In [34]:
df.loc[df.ForgivenessAmount.isnull(), 'ForgivenessAmount'] = 0

In [35]:
print(len(df[df.ForgivenessAmount.isnull()]))

0


## Export Cleaned Data (one file)

In [36]:
df.to_csv(cleaned_data_dir / 'export.csv', index=False)

## Generate Report

In [37]:
report_title = "PPP-FOIA Data Report"
report_filename = Path(f"{report_title.replace(' ', '_')}.html")
if not os.path.exists(report_filename):
    profile = ProfileReport(df, title=report_title)
    profile.to_file(report_filename)
else:
    print("Delete or rename existing report first!")
    print(report_filename)

Summarize dataset:  83%|███████████████████████▏    | 24/29 [06:52<00:30,  6.11s/it, Calculate auto correlation]C:\Users\knugg\Projects\ppp-loans-ds4a\.venv\Lib\site-packages\ydata_profiling\model\pandas\correlations_pandas.py:84: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  return _cramers_corrected_stat(pd.crosstab(col_1, col_2), correction=True)
C:\Users\knugg\Projects\ppp-loans-ds4a\.venv\Lib\site-packages\ydata_profiling\model\correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'cannot reindex on an axis with duplicate labels')
  warnings.warn(
Export report to file: 100%|███████████████████████████████████

## Transform Datatypes

dtypes can be applied when loading data. The following definition constains all columns, but requires field updates.

In [38]:
# TODO: Select correct dtypes and apply datetime transform as needed. Once ready, use these transforms to create/load the combined file.
dtype = {
    'LoanNumber': int,	                    # Loan Number (unique identifier)
    'DateApproved': str,	        # Loan Funded Date
    'SBAOfficeCode': str,	                # SBA Origination Office Code
    'ProcessingMethod': str,	            # Loan Delivery Method (PPP for first draw; PPS for second draw)
    'BorrowerName': str,	                # Borrower Name
    'BorrowerAddress': str,	                # Borrower Street Address
    'BorrowerCity': str,	                # Borrower City
    'BorrowerState': str,	                # Borrower State
    'BorrowerZip': str,	                    # Borrower Zip Code
    'LoanStatusDate': str,   	    # Loan Status Date - Loan Status Date is  blank when the loan is disbursed but not Paid In Full or Charged Off
    'LoanStatus': int,	                    # Loan Status Description - Loan Status is replaced by 'Exemption 4' when the loan is disbursed but not Paid in Full or Charged Off
    'Term': str,	                        # Loan Maturity in Months
    'SBAGuarantyPercentage': str,	        # SBA Guaranty Percentage
    'InitialApprovalAmount': str,	        # Loan Approval Amount(at origination)
    'CurrentApprovalAmount': str,	        # Loan Approval Amount (current)
    'UndisbursedAmount': str,	            # Undisbursed Amount
    'FranchiseName': str,	                # Franchise Name
    'ServicingLenderLocationID': str,	    # Lender Location ID (unique identifier)
    'ServicingLenderName': str,	            # Servicing Lender Name
    'ServicingLenderAddress': str,	        # Servicing Lender Street Address
    'ServicingLenderCity': str,     	    # Servicing Lender City
    'ServicingLenderState': str,	        # Servicing Lender State
    'ServicingLenderZip': str,	            # Servicing Lender Zip Code
    'RuralUrbanIndicator': str,	            # Rural or Urban Indicator (R/U)
    'HubzoneIndicator': str,	            # Hubzone Indicator (Y/N)
    'LMIIndicator': str,	                # LMI Indicator (Y/N)
    'BusinessAgeDescription': str,	        # Business Age Description
    'ProjectCity': str,	                    # Project City
    'ProjectCountyName': str,	            # Project County Name
    'ProjectState': str,	                # Project State
    'ProjectZip': str,	                    # Project Zip Code
    'CD': str,	                            # Project Congressional District
    'JobsReported': str,	                # Number of Employees
    'NAICSCode': str,	                    # NAICS 6 digit code
    'Race': str,	                        # Borrower Race Description
    'Ethnicity': str,               	    # Borrower Ethnicity Description
    'UTILITIES_PROCEED': str,	            # Note: Proceed data is lender reported at origination.  On the PPP application the proceeds fields were check boxes.  
    'PAYROLL_PROCEED': str,	                # Note: Proceed data is lender reported at origination.  On the PPP application the proceeds fields were check boxes.  
    'MORTGAGE_INTEREST_PROCEED': str,	    # Note: Proceed data is lender reported at origination.  On the PPP application the proceeds fields were check boxes.  
    'RENT_PROCEED': str,	                # Note: Proceed data is lender reported at origination.  On the PPP application the proceeds fields were check boxes.  
    'REFINANCE_EIDL_PROCEED': str,	        # Note: Proceed data is lender reported at origination.  On the PPP application the proceeds fields were check boxes.  
    'HEALTH_CARE_PROCEED': str,	            # Note: Proceed data is lender reported at origination.  On the PPP application the proceeds fields were check boxes.  
    'DEBT_INTEREST_PROCEED': str,	        # Note: Proceed data is lender reported at origination.  On the PPP application the proceeds fields were check boxes.  
    'BusinessType': str,	                # Business Type Description
    'OriginatingLenderLocationID': str,	    # Originating Lender ID (unique identifier)
    'OriginatingLender': str,	            # Originating Lender Name
    'OriginatingLenderCity': str,	        # Originating Lender City
    'OriginatingLenderState': str,	        # Originating Lender State
    'Gender': str,	                        # Gender Indicator
    'Veteran': str,	                        # Veteran Indicator
    'NonProfit': str,	                    # 'Yes' if Business Type = Non-Profit Organization or Non-Profit Childcare Center or 501(c) Non Profit
    'ForgivenessAmount': str,	            # Forgiveness Amount
    'ForgivenessDate': str,	                # Forgiveness Paid Date
}

## Validate Data (Check for Errors)